In [1]:
#General imports
from typing import Any
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import os
from collections import defaultdict 



# Lightning imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torch.optim import Adam, AdamW
from torch.utils.data import Dataset, DataLoader

#Lightning imports
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from lightning.pytorch.loggers import TensorBoardLogger
from torchmetrics import MetricCollection
from torchmetrics.regression import R2Score, MeanSquaredError, MeanAbsoluteError
from torchmetrics.classification import BinaryF1Score, BinaryAveragePrecision, BinaryAUROC, BinaryAccuracy
from pytorch_lightning.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping


#sklearn imports
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import roc_auc_score, precision_score, f1_score, accuracy_score, log_loss
from sklearn.model_selection import train_test_split, cross_val_score

#RF plus imports
import imodels
from imodels.tree.rf_plus.rf_plus.rf_plus_models import _RandomForestPlus, RandomForestPlusRegressor, RandomForestPlusClassifier
from imodels.tree.rf_plus.rf_plus_prediction_models.aloocv import AloGLM
from imodels.tree.rf_plus.rf_plus_prediction_models.aloocv_regression import AloElasticNetRegressorCV, AloLOL2Regressor
from imodels.tree.rf_plus.rf_plus_prediction_models.aloocv_classification import AloGLMClassifier, AloLogisticElasticNetClassifierCV, AloSVCRidgeClassifier
from imodels.tree.rf_plus.rf_plus.MOE.moe_utils import TabularDataset, TreePlusExpert, AloTreePlusExpert, GatingNetwork

#Testing imports
import openml
import time

#
from xgboost import XGBRegressor


In [2]:
from imodels.tree.rf_plus.rf_plus.MOE.rfplus_MOE import RandomForestPlusMOE

In [ ]:
#Load Data 
suite_id = 353
benchmark_suite = openml.study.get_suite(suite_id)
task_ids = benchmark_suite.tasks
task_id =  361235
random_state = 0
task = "regression"
seed_everything(random_state, workers=True)
print(f"Task ID: {task_id}")
task = openml.tasks.get_task(task_id)
dataset_id = task.dataset_id
dataset = openml.datasets.get_dataset(dataset_id)

# Split data into train, validation, and test sets
max_train = 500
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, y_train = copy.deepcopy(X_train)[:max_train], copy.deepcopy(y_train)[:max_train]
X_train_torch, X_val_torch, y_train_torch, y_val_torch = train_test_split(copy.deepcopy(X_train),copy.deepcopy(y_train), test_size=0.25)


#Get datasets and dataloaders
train_dataset = TabularDataset(torch.tensor(X_train_torch), torch.tensor(y_train_torch))
train_dataloader = DataLoader(train_dataset, batch_size=X_train_torch.shape[0])

val_dataset = TabularDataset(torch.tensor(X_val_torch), torch.tensor(y_val_torch))
val_dataloader = DataLoader(val_dataset, batch_size=X_val_torch.shape[0])

test_dataset = TabularDataset(torch.tensor(X_test), torch.tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0])

#fit RF plus model
if task == "classification":
    n_estimators = 256
    min_samples_leaf = 2
    max_epochs = 50
    max_features = "sqrt"
else:
    n_estimators = 256
    min_samples_leaf = 5
    max_epochs = 50
    max_features = 0.33

# rf_model = RandomForestClassifier(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf, max_features=max_features,random_state=random_state)
# rf_model.fit(X_train, y_train)
# rfplus_model = RandomForestPlusClassifier(rf_model = rf_model,fit_on = "all")
# rfplus_model.fit(X_train,y_train,n_jobs=-1)

rf_model = RandomForestRegressor(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf, max_features=max_features,random_state=random_state)
rf_model.fit(X_train, y_train)
rfplus_model = RandomForestPlusRegressor(rf_model = rf_model,fit_on = "all")
rfplus_model.fit(X_train,y_train,n_jobs=-1)

xgb_model = XGBRegressor(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf, max_features=max_features,random_state=random_state)
xgb_model.fit(X_train, y_train)

# # RFplus_MOEmodel = RandomForestPlusRegressor(rf_model=rf_model,fit_on = "all")  
# # RFplus_MOEmodel.fit(X_train,y_train,n_jobs=-1)

#Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(dirpath='checkpoints',filename='best_model',monitor='val_loss',mode='min',save_top_k=1,save_last=True,verbose=True)
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=5, verbose=False, mode="min")


RFplus_MOE = RandomForestPlusMOE(rfplus_model=rfplus_model, input_dim=X.shape[1], criterion= nn.MSELoss(), use_loo = False, train_experts=  True) #BinaryF1ScoreBinaryF1Score
logger = TensorBoardLogger(f'RFMOE_task_{task_id}', name='RFMOE')
trainer = Trainer(accelerator="cpu",max_epochs=max_epochs,callbacks=[checkpoint_callback],logger=logger)
trainer.fit(RFplus_MOE, train_dataloader, val_dataloader)
test = trainer.test(dataloaders=test_dataloader)


if rfplus_model._task == "classification":
    class_metrics = [accuracy_score, f1_score, precision_score]
    prob_metrics = [log_loss,roc_auc_score]
    for m in class_metrics:
        print(m.__name__)
        print("RF model: ",m(y_test,rf_model.predict(X_test)))
        print("XGB model: ",m(y_test,xgb_model.predict(X_test)))
        print("RF+ Model without MOE: ",m(y_test,rfplus_model.predict(X_test)))
        print("\n")
    for m in prob_metrics:
        print(m.__name__)
        print("RF model: ",m(y_test,rf_model.predict_proba(X_test)[:,1]))
        print("XGB model: ",m(y_test,xgb_model.predict_proba(X_test)[:,1]))
        print("RF+ Model without MOE: ",m(y_test,rfplus_model.predict_proba(X_test)[:,1]))
        print("\n")
else:
    metrics = [mean_absolute_error,mean_squared_error, r2_score]
    for m in metrics:
        print(m.__name__)
        print("RF model: ",m(y_test,rf_model.predict(X_test)))
        print("XGB model: ",m(y_test,xgb_model.predict(X_test)))
        print("RF+ Model without MOE: ",m(y_test,rfplus_model.predict(X_test)))
        print("\n")





Seed set to 0
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/tmp/ipykernel_2965104/3893806293.py:12:

Task ID: 361235


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 256 out of 256 | elapsed:   33.5s finished
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:59:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_features", "min_samples_leaf" } are not used.

  warnings.warn(smsg, UserWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/users/zachrewolinski/conda/envs/mdi/lib/pyt ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


...

Epoch 49, global step 50: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=50` reached.
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /accounts/grad/zachrewolinski/research/imodels-experiments/feature_importance/moe/checkpoints/best_model.ckpt
Loaded model weights from the checkpoint at /accounts/grad/zachrewolinski/research/imodels-experiments/feature_importance/moe/checkpoints/best_model.ckpt
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many w

Testing: |          | 0/? [00:00<?, ?it/s]

x.shape: torch.Size([301, 5])


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  test_MeanAbsoluteError   │    2.2748050689697266     │
│   test_MeanSquaredError   │     8.67469596862793      │
│       test_R2Score        │    0.8155949115753174     │
└───────────────────────────┴───────────────────────────┘

mean_absolute_error
RF model:  3.3847517866559436
XGB model:  1.6686230276114125
RF+ Model without MOE:  2.785857318691402


mean_squared_error
RF model:  17.687629293494645
XGB model:  5.630796183102364
RF+ Model without MOE:  12.230324268182796


r2_score
RF model:  0.6239795370723867
XGB model:  0.8802951739722457
RF+ Model without MOE:  0.7399961228965639




In [ ]:
# get the gating scores for each data point
gating_scores = RFplus_MOE.get_gating_scores(X_test)
